In [14]:
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/drive/MyDrive/datasetchatbot.json').read()
intents = json.loads(data_file)


In [20]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [21]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

942 documents
258 classes ['     Support Vector Machine', '   Decision Trees', '  Perceptron', '  Regularization in Machine Learning', '  applications of supervised learning', '  basic data structures', " 'action' in reinforcement learning", " 'agent' in reinforcement learning", " 'value' in reinforcement learning", ' Classification model', ' Gradient Descent', ' Inductive Logic Programming', ' Polynomial Regression', ' Regularization', ' Ridge Regression technique', ' Types of machine learning', ' advantages of using Polynomial Regression over Linear Regression', ' applications of self-supervised learning', ' components of Bayesian logic program', ' difference between heuristic for rule learning and heuristics', ' disadvantages of supervised learning', ' imbalanced datasheet', ' issues to consider in supervised learning', ' standard supervised learning', 'A/B Testing', 'Algorithm techniques in Machine Learning', 'Appreciation', 'Apriori algorithm', 'Areas of Problems', 'BiasML', 'Bina

In [22]:
pickle.dump(words,open('words(01).pkl','wb'))
pickle.dump(classes,open('classes(01).pkl','wb'))

In [23]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]
print("Training data created")

Training data created


In [24]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

# Create the model
model = Sequential()
if train_x:  # Check if train_x is not empty
    model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
else:
    print("Error: train_x is empty. Check data loading and preprocessing.")

model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

if train_y:  # Check if train_y is not empty
    model.add(Dense(len(train_y[0]), activation='softmax'))
else:
    print("Error: train_y is empty. Check data loading and preprocessing.")

# Compile the model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(np.array(train_x), np.array(train_y), verbose=0)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Save the model as an .h5 file
model.save('model_name.h5')



Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0046 - loss: 5.5585
Epoch 2/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0268 - loss: 5.5009
Epoch 3/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0285 - loss: 5.4320
Epoch 4/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0227 - loss: 5.3218
Epoch 5/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0331 - loss: 5.1648
Epoch 6/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0506 - loss: 4.8951
Epoch 7/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0887 - loss: 4.6290
Epoch 8/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0991 - loss: 4.4176
Epoch 9/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1370 - loss: 3.9642
Epoch 10/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1564 - loss: 3.9043
Epoch 11/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1635 - loss: 3.6116
Epoch 12/200
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Accuracy: 97.88%
